In [19]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [20]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../resources/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

(250000, 30)


## Do your thing crazy machine learning thing here :) ...

In [23]:
import implementations as imp
imp.leastsquaresGD(y,tX,np.zeros(tX.shape[1]),20,0.7)

(5.27896549454422e+249,
 array([-6.37930228e+126,  3.20951389e+126,  4.83666900e+126,
         1.86178365e+126, -6.00829278e+127, -6.10431901e+127,
        -6.00738245e+127,  1.50396509e+125,  8.62267799e+125,
         5.88898891e+126,  8.31642064e+124, -2.95754864e+124,
        -6.00790372e+127,  2.08940978e+126, -9.58480689e+122,
        -7.46914939e+122,  2.58258592e+126, -1.79458232e+123,
         2.56460694e+123,  2.01727388e+126, -1.04938868e+123,
         8.89038188e+126,  2.00320540e+124, -3.59259430e+127,
        -3.72490183e+127, -3.72493429e+127, -6.01619209e+127,
        -6.00782195e+127, -6.00782313e+127,  1.21699318e+126]))

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)